In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [86]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [87]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
720,(2022/11/30) 23-24,88,0
721,Min,56,0
722,Max,310.19,20
723,Average,124.73073611111111111111111111,6.5277777777777777777777777778


In [88]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
716,(2022/11/30) 19-20,240.00,0.0
717,(2022/11/30) 20-21,96.00,0.0
718,(2022/11/30) 21-22,96.00,0.0
719,(2022/11/30) 22-23,88.00,0.0


In [89]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
716,(2022/11/30) 19-20,240.00,0.0,2022-11-30,19
717,(2022/11/30) 20-21,96.00,0.0,2022-11-30,20
718,(2022/11/30) 21-22,96.00,0.0,2022-11-30,21
719,(2022/11/30) 22-23,88.00,0.0,2022-11-30,22


In [90]:
dat=dat.drop(['datehour'],1)
dat

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\4059115739.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dat=dat.drop(['datehour'],1)


,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
716,240.00,0.0,2022-11-30,19
717,96.00,0.0,2022-11-30,20
718,96.00,0.0,2022-11-30,21
719,88.00,0.0,2022-11-30,22


In [91]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2022-11-30,240.00,0.0,19,2022,11,30,2
2022-11-30,96.00,0.0,20,2022,11,30,2
2022-11-30,96.00,0.0,21,2022,11,30,2


In [92]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2022 11    26      0.0      0.0              NaN      96.00       75.0   
           27      0.0      0.0              NaN      75.00       75.0   
           28      0.0      0.0              NaN     240.00       88.0   
           29      0.0      0.0              NaN     240.00       88.0   
           30      0.0      0.0              NaN     240.00       88.0   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  
           7     59.710417       50.0        0.0   16.666667 2017-03-07  
           8     59.710417       50.0        0.0   16.666667 2017-03-08  
           9     59.710417       50.0        0.0   16.666667 2017-03-09  
           10    65.852500       50.0        0.0   10.000000 2017-03-10  
...                    ...        ...        ...         ...        ...  
2022 11    26    81.125000        0.0        0.0    0.000000 2022-11-26  
           27    75.000000        0.0        0.0    0.000000 2022-11-27  
           28   123.333333        0.0        0.0    0.000000 2022-11-28  
           29   130.400000        0.0        0.0    0.000000 2022-11-29  
           30   130.400000        0.0        0.0    0.000000 2022-11-30  

[2042 rows x 10 columns]

In [93]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,morning
...,...,...,...,...,...,...,...,...,...
2022-11-30,240.00,0.0,19,2022,11,30,2,0.0,evening
2022-11-30,96.00,0.0,20,2022,11,30,2,0.0,off
2022-11-30,96.00,0.0,21,2022,11,30,2,0.0,off


In [94]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2022 11    26      0.0      0.0              NaN      96.00       75.0   
           27      0.0      0.0              NaN      75.00       75.0   
           28      0.0      0.0              NaN     240.00       88.0   
           29      0.0      0.0              NaN     240.00       88.0   
           30      0.0      0.0              NaN     240.00       88.0   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2017 3     6     65.852500       50.0        0.0   10.000000 2017-03-06  ...   
           7     59.710417       50.0        0.0   16.666667 2017-03-07  ...   
           8     59.710417       50.0        0.0   16.666667 2017-03-08  ...   
           9     59.710417       50.0        0.0   16.666667 2017-03-09  ...   
           10    65.852500       50.0        0.0   10.000000 2017-03-10  ...   
...                    ...        ...        ...         ...        ...  ...   
2022 11    26    81.125000        0.0        0.0    0.000000 2022-11-26  ...   
           27    75.000000        0.0        0.0    0.000000 2022-11-27  ...   
           28   123.333333        0.0        0.0    0.000000 2022-11-28  ...   
           29   130.400000        0.0        0.0    0.000000 2022-11-29  ...   
           30   130.400000        0.0        0.0    0.000000 2022-11-30  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 3     6                   3.0                 2.0       150.0   
           7                   3.0                 2.0       350.0   
           8                   3.0                 2.0       350.0   
           9                   3.0                 2.0       350.0   
           10                  3.0                 2.0       150.0   
...                            ...                 ...         ...   
2022 11    26                  NaN                 NaN         0.0   
           27                  NaN                 NaN         0.0   
           28                  3.0                 2.0         0.0   
           29                  3.0                 2.0         0.0   
           30                  3.0                 2.0         0.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 3     6               50.0             0.0             0.0     4350.0   
           7               50.0             0.0             0.0    10150.0   
           8               50.0             0.0             0.0    10150.0   
           9               50.0             0.0             0.0    10150.0   
           10              50.0             0.0             0.0     4350.0   
...                         ...             ...             ...        ...   
2022 11    26               0.0             NaN             NaN        0.0   
           27               NaN             NaN             NaN        0.0   
           28               0.0             0.0             0.0        0.0   
           29               0.0             0.0             0.0        0.0   
           30               0.0             0.0             0.0        0.0   

                standard_value  morning_value  evening_value  
year month day         

In [95]:
daily.to_csv(out + pref + '_daily.csv')

In [96]:
hourly.to_csv(out + pref + '_hourly.csv')

In [97]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2022-11-30,19,240.00,0.0,evening
2022-11-30,20,96.00,0.0,off
2022-11-30,21,96.00,0.0,off


In [98]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [99]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\3018208690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)


,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,480.730769,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.588812,33.946194,61.625498,86.514416,115.321589,24.752613,11.700000,19.868778,7.550000
4,2017,7,462.677419,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,20.210081,...,42.0,44.364721,32.046585,59.895533,79.061769,110.973696,24.497238,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2022,7,220.000000,7758.664516,114.667500,30.08,83.517497,20.000000,3.225806,9.473118,...,42.0,35.266657,35.325875,34.364286,NaN,NaN,17.679558,0.000000,1.714286,0.000000
65,2022,8,287.267742,16564.169548,132.016000,31.02,95.694403,22.903226,4.883871,12.601210,...,46.0,57.661085,35.464299,85.102689,193.989240,270.040000,18.964602,5.050725,7.475969,4.326087
66,2022,9,409.333333,19963.273333,122.705000,27.05,87.623644,24.000000,6.000000,17.641667,...,44.0,48.770212,35.440447,73.627959,79.617778,NaN,24.101796,5.454545,15.604839,0.000000
67,2022,10,369.938710,17033.723806,124.000000,29.07,95.055970,33.570968,5.809677,16.067339,...,42.0,46.044719,45.556737,52.060186,NaN,NaN,29.302762,0.000000,3.512245,0.000000


In [100]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,480.730769,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.588812,33.946194,61.625498,86.514416,115.321589,24.752613,11.700000,19.868778,7.550000
4,2017,7,462.677419,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,20.210081,...,42.0,44.364721,32.046585,59.895533,79.061769,110.973696,24.497238,10.317460,18.755102,5.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2022,7,220.000000,7758.664516,114.667500,30.08,83.517497,20.000000,3.225806,9.473118,...,42.0,35.266657,35.325875,34.364286,NaN,NaN,17.679558,0.000000,1.714286,0.000000
65,2022,8,287.267742,16564.169548,132.016000,31.02,95.694403,22.903226,4.883871,12.601210,...,46.0,57.661085,35.464299,85.102689,193.989240,270.040000,18.964602,5.050725,7.475969,4.326087
66,2022,9,409.333333,19963.273333,122.705000,27.05,87.623644,24.000000,6.000000,17.641667,...,44.0,48.770212,35.440447,73.627959,79.617778,NaN,24.101796,5.454545,15.604839,0.000000
67,2022,10,369.938710,17033.723806,124.000000,29.07,95.055970,33.570968,5.809677,16.067339,...,42.0,46.044719,45.556737,52.060186,NaN,NaN,29.302762,0.000000,3.512245,0.000000


In [101]:
monthly=monthly.set_index(['year','month'])

In [102]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [103]:
monthly.to_csv(out + pref + '_monthly.csv')

In [104]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [105]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\2677402245.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)


,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,13070.766613,47.716782,111.316903,37.184000,66.971691,19.912258,4.410968,11.909199,44.375131,...,76.400000,405193.765000,348.200000,252.800000,66.6,44.400000,16.125080,9.574360,4.314148,1.669317
1,2,9487.851246,56.821624,124.906343,41.167537,78.819210,14.085821,2.097015,7.477313,51.278051,...,0.000000,254274.413400,311.000000,218.000000,57.0,38.000000,11.148002,4.803140,1.376111,0.000000
2,3,9375.628391,44.921459,124.015517,40.972299,73.281647,18.200000,3.419540,9.701767,48.594815,...,33.666667,271893.223333,325.000000,238.000000,63.0,42.000000,18.281671,4.832581,2.020063,1.122222
3,4,13000.576818,41.082873,106.156818,36.257403,62.852558,21.198701,5.056494,12.814659,42.243192,...,188.233333,333681.471667,291.333333,209.000000,55.0,36.666667,22.878275,13.194990,12.443687,12.139141
4,5,17622.612983,39.854333,98.518068,31.217670,56.866397,29.732955,8.563636,18.250682,37.027665,...,501.800000,516929.980833,333.666667,237.500000,62.5,41.666667,24.498520,14.758634,14.172815,14.617090
5,6,18457.284591,42.158900,128.855625,33.979261,69.907651,35.486364,6.700568,19.494711,41.943226,...,52.500000,541413.681333,331.333333,240.833333,63.5,42.333333,30.636064,10.768626,3.878331,1.309921
6,7,16472.733269,44.556346,144.999839,37.166613,78.788192,29.140323,5.445161,15.343051,46.843002,...,142.833333,510654.731333,350.500000,251.500000,66.0,44.000000,25.846831,6.692935,3.810156,3.287698
7,8,19195.023344,52.205692,151.115108,36.657097,77.791917,27.290323,6.402151,16.171304,45.232459,...,310.833333,595045.723667,346.666667,253.666667,67.0,44.666667,22.836203,10.839196,7.772508,6.757246
8,9,23234.718883,48.898047,124.431167,32.590444,66.834794,37.264444,7.773333,21.146153,40.220133,...,319.166667,697041.566500,341.666667,243.666667,64.0,42.666667,30.529284,13.456302,9.282744,7.901389
9,10,16353.764554,47.591831,109.032742,35.598925,66.579042,21.795699,7.056989,14.778929,43.542671,...,178.333333,506966.701167,350.500000,251.500000,66.0,44.000000,19.815495,11.775993,6.521410,4.021739


In [106]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [107]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\1619064361.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)


price     volume        value
month hour                                    
1     0      37.320522  17.150435   664.675922
      1      37.320522  17.150435   664.675922
      2      37.320522  17.150435   664.675922
      3      37.320522  17.150435   664.675922
      4      37.184000  17.017419   657.171090
...                ...        ...          ...
12    19    126.787806  12.840000   725.605355
      20     71.233355  21.669032  1190.360852
      21     71.233355  21.669032  1190.360852
      22     38.137871  42.378710  1755.612090
      23     38.137871  42.378710  1755.612090

[288 rows x 3 columns]

In [108]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [109]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\4270116260.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)


price     volume        value
year month hour                                    
2017 3     0      27.422941  50.000000  1323.629412
           1      27.422941  50.000000  1323.629412
           2      27.422941  50.000000  1323.629412
           3      27.422941  50.000000  1323.629412
           4      27.588947  50.000000  1336.931579
...                     ...        ...          ...
2022 11    19    215.331000   2.000000   177.320000
           20    110.465333   4.000000   354.640000
           21    110.465333   4.000000   354.640000
           22     86.495333  13.333333  1203.906667
           23     86.495333  13.333333  1203.906667

[1656 rows x 3 columns]

In [110]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [111]:
from statsmodels.tsa.stattools import pacf

In [112]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.716986,0.740439,0.743976,0.675430
2,-0.063935,-0.197671,-0.179618,-0.079544
3,0.233793,0.240730,0.250364,0.177195
4,0.195866,0.152133,0.172703,0.085858
5,0.060224,0.132853,0.123028,0.052417
6,0.124685,0.144633,0.181569,0.144527
7,0.119121,-0.072368,-0.058875,0.189571
8,-0.007645,0.086280,0.074216,0.061551
9,-0.280167,-0.251118,-0.272525,-0.305701


In [113]:
pacfs.to_csv(out + pref + '_pacfs.csv')